In [8]:
import pandas as pd, numpy as np, json, os

In [9]:
m=json.loads(open('json/member_timelines.json','r').read())

In [10]:
data={'Ország':{},'Bizottság':{},'Párt':{}}
attention={}
for k,i in enumerate(m):
    if k%1000==0: print(k/len(m)*100,'%')
    if i['Típus'] in list(data.keys()):
        if i['Csoportok'] not in data[i['Típus']]:data[i['Típus']][i['Csoportok']]={}
        for t in pd.date_range(i['start'],i['end']):
            st=str(t)[:10]
            if st not in data[i['Típus']][i['Csoportok']]:data[i['Típus']][i['Csoportok']][st]=set()
            data[i['Típus']][i['Csoportok']][st].add(i['Name'])
            
            if i['Name'] not in attention:attention[i['Name']]={}
            if st not in attention[i['Name']]:attention[i['Name']][st]=0
            attention[i['Name']][st]+=1

0.0 %
3.289040915668991 %
6.578081831337982 %
9.867122747006974 %
13.156163662675963 %
16.445204578344956 %
19.734245494013948 %
23.023286409682935 %
26.312327325351927 %
29.60136824102092 %
32.89040915668991 %
36.1794500723589 %
39.468490988027895 %
42.75753190369688 %
46.04657281936587 %
49.33561373503486 %
52.62465465070385 %
55.913695566372844 %
59.20273648204184 %
62.49177739771083 %
65.78081831337983 %
69.06985922904882 %
72.3589001447178 %
75.64794106038678 %
78.93698197605579 %
82.22602289172477 %
85.51506380739376 %
88.80410472306276 %
92.09314563873174 %
95.38218655440073 %
98.67122747006972 %


In [339]:
data2=dict(data)
for tipus in data:
    for csoport in data[tipus]:
        for t in data[tipus][csoport]:
            data2[tipus][csoport][t]=list(data[tipus][csoport][t])

In [340]:
#save
open('E:/Onedrive - Lancaster University/datarepo/influence/ro/data/member_pairs.json','w').write(json.dumps(data2))

536767417

In [2]:
#load
# data=json.loads(open('E:/Onedrive - Lancaster University/datarepo/influence/ro/data/member_pairs.json','r').read())
data=json.loads(open('C:/Users/csala/Onedrive - Lancaster University/datarepo/influence/ro/data/member_pairs.json','r').read())

In [ ]:
network={}
tnetwork={}
weights={'Ország':2,'Bizottság':3,'Párt':1}
attention_penalty=0.5 #parameter that controls the attention penalty. larger values take the advantage from people in larger groups than their average
group_penalty=0.5 #parameter that control the group penalty. larger values increase the large group penalty effect
cohort_attention_penalty=0.25 #parameter that controls the cohort attention penalty. larger values take the advantage from people who are in many groups
cohort_mean_attention=np.mean([np.mean(list(attention[name].values())) for name in attention]) #cohort mean of individual mean attentions
for tipus in data:
    for csoport in data[tipus]:
        print(csoport)
        for t in data[tipus][csoport]:
            names=list(data[tipus][csoport][t])
            
            #we also track influence by year
            #st=t[:7]
            st=t[:4]
            if st not in tnetwork:tnetwork[st]={}
            
            #more people in the group reduce your ability to make friends
            group_size=len(names)
            relative_group=(5/group_size)**group_penalty
            #relative_group=np.log(5/group_size)
            
            #logarithmic time anneal
            #anneal=2-float((pd.to_datetime('2019-05-01')-pd.to_datetime(t))/(pd.to_datetime('2019-05-01')-pd.to_datetime('1990-01-01')))**0.5
            anneal=max(0.,-np.log(float((pd.to_datetime('2019-05-01')-pd.to_datetime(t))/(pd.to_datetime('2019-05-01')-pd.to_datetime('1989-01-01')))))
            for r in range(len(names)):
                name=names[r]
                
                #the more gorups you are in the less likely you are able to pay attention
                #but of course everybody has a different basic attention level, and basic socialization level, number of assistents, etc.
                #so we take the relative situation of the current number of groups versus usually the person is in how many groups
                #we call this relative attention
                mean_attention=np.mean(list(attention[name].values()))
                relative_attention=(mean_attention/attention[name][t])**attention_penalty
                #relative_attention=np.log(mean_attention/attention[name][t])
                
                #one could argue to normalize for this inside the cohort, but a larger team might actually mean an extention of the influence, so we chose a weak effect
                cohort_relative_attention=(cohort_mean_attention/mean_attention)**cohort_attention_penalty
                #cohort_relative_attention=np.log(cohort_mean_attention/mean_attention)
                
                if name not in network: network[name]={}
                if name not in tnetwork[st]: tnetwork[st][name]={}
                
                #asymmetric network
                for p in range(len(names)):
                    name2=names[p]
                    if name!=name2:
                        v=weights[tipus]*anneal*relative_group*relative_attention*cohort_relative_attention
                        if name2 not in network[name]: network[name][name2]=0
                        network[name][name2]+=v
                        if name2 not in tnetwork[st][name]: tnetwork[st][name][name2]=0
                        tnetwork[st][name][name2]+=v

In [233]:
open('C:/Users/csala/Onedrive - Lancaster University/datarepo/influence/ro/data/member_pagerank_raw_log.json','w').write(json.dumps(network))

46850855

In [ ]:
#load
#network=json.loads(open('C:/Users/csala/Onedrive - Lancaster University/datarepo/influence/ro/data/member_pagerank_raw_log.json','r').read())

In [234]:
open('C:/Users/csala/Onedrive - Lancaster University/datarepo/influence/ro/data/member_pagerank_raw_log.json','w').write(json.dumps(tnetwork))

173735510

Format for D3plus

Normalize to fraction of all committee days spent together

In [284]:
network.keys()

dict_keys(['2012', '2013', '2014', '2015', '1990', '1991', '1992', '2010', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2016', '2017', '2018', '2019', '2008', '2009', '2005', '2006', '2007', '2011', '1996', '1993', '1994', '1995'])

In [283]:
members_dict={}
members_set={}
for iname,name in enumerate(network):
    if iname%100==0: print(iname/len(network)*100,'%')
    for name2 in network[name]:
        if name!=name2:
            ratio=network[name][name2]/np.mean(list(network[name].values()))
            
            if name not in members_dict:members_dict[name]={}
            members_dict[name][name2]=ratio
            
            nset=str(set([name,name2]))
            if nset not in members_set:members_set[nset]=0
            members_set[nset]+=ratio

0.0 %


TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [279]:
len(members_dict),len(members_set)

(16, 35)

In [280]:
member_rings_normed=[]
all_names=set()
min_ratio=25
for iname,name in enumerate(members_set):
    if iname%100000==0: print(iname/len(members_set)*100,'%')
    names=[i.strip() for i in name.replace('{','').replace('}','').replace("'",'').split(',')]
    ratio=members_set[name]
    if ratio>min_ratio:
        all_names.add(names[0])
        all_names.add(names[1])
        member_rings_normed.append({'source':names[0],'target':names[1],'ratio':ratio})

0.0 %


In [281]:
print(len(member_rings_normed))

0


In [266]:
#add lone nodes
member_rings_normed3=member_rings_normed+[{'source':i,'target':'Nincsenek erős kapcsolatok '+i,'ratio':1} for i in network if i not in all_names]

Network analysis

In [267]:
import networkx as nx

In [272]:
# G=nx.Graph(members_dict) 
#G=nx.Graph(network) 
G = nx.Graph(pd.DataFrame(member_rings_normed3), source='source', target='target') 
pr = nx.pagerank(G)
sorted_pr = sorted(pr.items(), key=lambda kv: kv[1], reverse=True)
member_rings_nodes={i[0]:i[1] for i in sorted_pr}
member_rings_nodes={i:[{'id':'','Befolyás':member_rings_nodes[i]/np.mean(list(member_rings_nodes.values()))}] for i in member_rings_nodes}

In [269]:
tmember_rings_nodes=[]
for t in tnetwork:
    print(t)
    G=nx.Graph(tnetwork[t]) 
    pr = nx.pagerank(G)
    sorted_pr = sorted(pr.items(), key=lambda kv: kv[1], reverse=True)
    smember_rings_nodes={i[0]:i[1] for i in sorted_pr}
    tmember_rings_nodes+=[{'id':i,'x':t,'y':smember_rings_nodes[i]/np.mean(list(smember_rings_nodes.values()))} for i in smember_rings_nodes]

2012
2013
2014
2015
1990
1991
1992
2010
1997
1998
1999
2000
2001
2002
2003
2004
2016
2017
2018
2019
2008
2009
2005
2006
2007
2011
1996
1993
1994
1995


In [270]:
tmember_rings_dict={}
for i in tmember_rings_nodes:
    if i['id'] not in tmember_rings_dict:tmember_rings_dict[i['id']]=[]
    z=dict(i)
    z['id']='Relatív befolyás'
    #z['x']=int(z['x'])
    #z['x']=str(pd.to_datetime(z['x']))
    tmember_rings_dict[i['id']].append(z)

In [273]:
open('json/member_pagerank_normed.json','w').write(json.dumps({'nodes_abs':member_rings_nodes,'nodes_t':tmember_rings_dict,'edges':member_rings_normed3}))

2325318